In [1]:
import sys
from sys import exit
import glob
import numpy as np
import pandas as pd
from os.path import join, isfile
from datetime import date, datetime
from unidecode import unidecode

from xlsxwriter.worksheet import (
    Worksheet, cell_number_tuple, cell_string_tuple)
    
from time import time, sleep
from contextlib import contextmanager
import functools
import threading

from my_funcs import ftime_elapsed, normalize_number, normalize_text, normalize_hash, trim_overspace, isvaliddate, normalize_cpf, normalize_date, date_hash, get_better, get_better_2, fit_cols, compare_dates, get_casos_confirmados, get_vacinados, get_tb_vacinados


pd.set_option('display.max_columns', 0)
pd.options.display.float_format = lambda x: '{:,.0f}'.format(x).replace(',','.')

meses = ['Janeiro','Fevereiro','Março','Abril','Maio','Junho','Julho','Agosto','Setembro','Outubro','Novembro','Dezembro']
municipios = pd.read_csv('municipios.csv',converters={'ibge':str})
regionais = pd.read_csv('regionais.csv',converters={'ibge':str})

# faixa_etaria = [0,30,40,50,60,70,80,90,100]
# faixa_etaria_labels = ['err','0-29','30-39','40-49','50-59','60-69','70-79','80-89','90-99','>=100']
# faixa_etaria = [60,65,70,75,80,85,90]
# faixa_etaria_labels = ['0-59','60-64','65-69','70-74','75-79','80-84','85-89','>=90']
faixa_etaria = [60,70,80,90]
faixa_etaria_labels = ['0-59','60-69','70-79','80-89','>=90']

In [ ]:
init_time = time()
vacinados = get_vacinados(load=True)
print(ftime_elapsed(init_time), 'vacinados shape:', vacinados.shape)
vacinados['mes_aplicacao'] = vacinados['data_aplicacao'].apply(lambda x: meses[x.month-1] if isinstance(x,date) else 'erro')
vacinados.groupby('mes_aplicacao')[['sexo']].count()

In [ ]:
vacinados.columns

In [ ]:
vacinados.groupby('dose')[['sexo']].count()

In [ ]:
doses_header = ['cns','cpf','paciente','sexo','nome_mae','data_nascimento','ibge','data_aplicacao','fabricante','vacina_nome','hash_mae','hash_nasc'] #
primeira_dose = vacinados.loc[(vacinados['dose']=='1ª Dose')|(vacinados['dose']=='Única'),doses_header]
segunda_dose = vacinados.loc[vacinados['dose']=='2ª Dose',doses_header]

tb_vacinados = pd.merge(primeira_dose,segunda_dose, how='outer',on='cns',suffixes=['_1a_dose','_2a_dose'])

tb_vacinados['cpf'] = tb_vacinados.apply(lambda x: get_better(x['cpf_1a_dose'],x['cpf_2a_dose']), axis=1)
tb_vacinados['data_nascimento'] = tb_vacinados.apply(lambda x: get_better(x['data_nascimento_1a_dose'],x['data_nascimento_2a_dose'],datetime), axis=1)
tb_vacinados['nome_mae'] = tb_vacinados.apply(lambda x: get_better(x['nome_mae_1a_dose'],x['nome_mae_2a_dose']), axis=1)
tb_vacinados['paciente'] = tb_vacinados.apply(lambda x: get_better(x['paciente_1a_dose'],x['paciente_2a_dose']), axis=1)
tb_vacinados['fabricante'] = tb_vacinados.apply(lambda x: get_better(x['fabricante_1a_dose'],x['fabricante_2a_dose']), axis=1)
tb_vacinados['vacina_nome'] = tb_vacinados.apply(lambda x: get_better_2(x['vacina_nome_1a_dose'],x['vacina_nome_2a_dose']), axis=1)
tb_vacinados['hash_mae'] = tb_vacinados.apply(lambda x: get_better(x['hash_mae_1a_dose'],x['hash_mae_2a_dose']), axis=1)
tb_vacinados['hash_nasc'] = tb_vacinados.apply(lambda x: get_better(x['hash_nasc_1a_dose'],x['hash_nasc_2a_dose']), axis=1)
tb_vacinados['ibge'] = tb_vacinados.apply(lambda x: get_better(x['ibge_1a_dose'],x['ibge_2a_dose']), axis=1)
tb_vacinados['sexo'] = tb_vacinados.apply(lambda x: get_better(x['sexo_1a_dose'],x['sexo_2a_dose']), axis=1)

tb_vacinados = tb_vacinados.drop(columns=['cpf_1a_dose','cpf_2a_dose', 'data_nascimento_1a_dose', 'data_nascimento_2a_dose', 'paciente_1a_dose', 'paciente_2a_dose', 'fabricante_1a_dose', 'fabricante_2a_dose', 'vacina_nome_1a_dose', 'vacina_nome_2a_dose', 'nome_mae_1a_dose', 'nome_mae_2a_dose', 'hash_mae_1a_dose', 'hash_mae_2a_dose', 'hash_nasc_1a_dose', 'hash_nasc_2a_dose','ibge_1a_dose','ibge_2a_dose','sexo_2a_dose','sexo_1a_dose'])

tb_vacinados.loc[(tb_vacinados['data_aplicacao_1a_dose'].notnull()) & (tb_vacinados['data_aplicacao_2a_dose'].notnull()), 'dias_entre_doses'] = (
    tb_vacinados.loc[(tb_vacinados['data_aplicacao_1a_dose'].notnull()) & (tb_vacinados['data_aplicacao_2a_dose'].notnull()), 'data_aplicacao_2a_dose'] -
    tb_vacinados.loc[(tb_vacinados['data_aplicacao_1a_dose'].notnull()) & (tb_vacinados['data_aplicacao_2a_dose'].notnull()), 'data_aplicacao_1a_dose'])

tb_vacinados.loc[tb_vacinados['dias_entre_doses'].notnull(),'dias_entre_doses'] = tb_vacinados.loc[tb_vacinados['dias_entre_doses'].notnull(),'dias_entre_doses'].apply(lambda x: int(x.days))

tb_vacinados.loc[tb_vacinados['dias_entre_doses'].isnull(),'dias_entre_doses'] = 0

tb_vacinados['situacao'] = 'inconsistencia'

tb_vacinados.loc[(tb_vacinados['data_aplicacao_1a_dose'].notnull()) & (tb_vacinados['data_aplicacao_2a_dose'].isnull()), 'situacao'] = 'só primeira dose'

tb_vacinados.loc[(tb_vacinados['dias_entre_doses']>0), 'situacao'] = 'ambas doses aplicadas'

tb_vacinados['data_ultima_dose'] = tb_vacinados['data_aplicacao_1a_dose'].copy()
tb_vacinados.loc[tb_vacinados['data_aplicacao_2a_dose'].notna(),'data_ultima_dose'] = tb_vacinados.loc[tb_vacinados['data_aplicacao_2a_dose'].notna(),'data_aplicacao_2a_dose'].copy()

tb_vacinados['dias_apos_ultima_dose'] = tb_vacinados.apply(lambda x: (pd.to_datetime(date.today()) - x['data_ultima_dose']).days, axis=1)

vacinados['ibge'] = vacinados['ibge'].fillna('999999').apply(str)
regionais['ibge'] = regionais['ibge'].apply(str)

tb_vacinados = pd.merge(tb_vacinados,regionais[['ibge','nu_reg','nm_reg','nm_macro']],on='ibge',how='left')

tb_vacinados.loc[(tb_vacinados['data_nascimento'].notnull()) & (tb_vacinados['data_ultima_dose'].notnull()), 'idade'] = \
    tb_vacinados.loc[(tb_vacinados['data_nascimento'].notnull()) & (tb_vacinados['data_ultima_dose'].notnull())].apply(
            lambda row: row['data_ultima_dose'].year - row['data_nascimento'].year - (
                    (row['data_ultima_dose'].month, row['data_ultima_dose'].day) <
                    (row['data_nascimento'].month, row['data_nascimento'].day)
            ), axis=1
    )
tb_vacinados.loc[tb_vacinados['data_nascimento'].isna(), 'idade'] = -99
tb_vacinados['idade'] = tb_vacinados['idade'].apply(int)

tb_vacinados['faixa_etaria'] = [ faixa_etaria_labels[idx] for idx in np.digitize(tb_vacinados['idade'],faixa_etaria,right=False)]

print(len(tb_vacinados))
 
print(len(tb_vacinados.loc[tb_vacinados.duplicated("cns",keep="first")]))
tb_vacinados = tb_vacinados.drop_duplicates("cns", keep="first")

print(len(tb_vacinados.loc[tb_vacinados.duplicated("cpf",keep="first")]))
tb_vacinados = tb_vacinados.drop_duplicates("cpf", keep="first")

print(len(tb_vacinados.loc[tb_vacinados.duplicated("hash_mae",keep="first")]))
tb_vacinados = tb_vacinados.drop_duplicates("hash_mae", keep="first")

print(len(tb_vacinados.loc[tb_vacinados.duplicated("hash_nasc",keep="first")]))
tb_vacinados = tb_vacinados.drop_duplicates("hash_nasc", keep="first")

print(len(tb_vacinados))

tb_vacinados.groupby('faixa_etaria')[['cns']].count()

In [ ]:
bins = [1,2,3,7,14,21,30,60,90]
bins_label = ['hoje','24 horas','48 horas', '72 horas', '7 dias', '14 dias', '21 dias', '30 dias', '60 dias', '90 dias']
tb_vacinados['periodo_ultima_dose'] = [ x for x in np.digitize(tb_vacinados['dias_apos_ultima_dose'],bins,right=False)]
dias_apos_ultima_dose = tb_vacinados.groupby(['situacao','periodo_ultima_dose'])[['cns']].count().unstack().fillna(0).astype(int).rename(columns={'cns':'qtde'}).droplevel(0,1)
dias_apos_ultima_dose['total'] = dias_apos_ultima_dose.sum(1)
dias_apos_ultima_dose = dias_apos_ultima_dose.append(pd.DataFrame(dias_apos_ultima_dose.sum().tolist(),index=dias_apos_ultima_dose.columns.tolist(),columns=['total']).T)
dias_apos_ultima_dose.columns = [ bins_label[bins] if isinstance(bins,int) else bins for bins in dias_apos_ultima_dose.columns ]
dias_apos_ultima_dose = dias_apos_ultima_dose.reset_index()
dias_apos_ultima_dose.index = [0,2,1,3]
dias_apos_ultima_dose = dias_apos_ultima_dose.sort_index().set_index('index')
dias_apos_ultima_dose.index.name = None
dias_apos_ultima_dose

In [ ]:
tb_vacinados['periodo_ultima_dose'] = tb_vacinados['periodo_ultima_dose'].apply(lambda x: bins_label[x]).copy()
tb_vacinados = tb_vacinados[['ibge','nu_reg','nm_reg','nm_macro','cns', 'cpf', 'paciente', 'sexo', 'data_nascimento', 'idade', 'faixa_etaria', 'nome_mae', 'vacina_nome', 'fabricante', 'data_aplicacao_1a_dose', 'data_aplicacao_2a_dose', 'dias_entre_doses', 'situacao', 'dias_apos_ultima_dose', 'periodo_ultima_dose', 'data_ultima_dose', 'hash_mae', 'hash_nasc']]
# tb_vacinados

In [ ]:
tb_vacinados = tb_vacinados.reset_index()
tb_vacinados.to_feather('tb_vacinados.feather')

# writer = pd.ExcelWriter(f"tb_vacinados.xlsx",
#                         engine='xlsxwriter',
#                         datetime_format='dd/mm/yyyy',
#                         date_format='dd/mm/yyyy')

# dias_apos_ultima_dose.to_excel(writer,'dias_apos_ultima_dose')
# fit_cols(writer,dias_apos_ultima_dose,'dias_apos_ultima_dose')

# tb_vacinados['mes_ultima_dose'] = tb_vacinados['data_ultima_dose'].apply(lambda x: meses[x.month])

# for mes, df in tb_vacinados.groupby('mes_ultima_dose'):
#     df.to_excel(writer,mes)
#     fit_cols(writer,df,mes)

# writer.save()

In [2]:
tb_vacinados = pd.read_feather('tb_vacinados.feather')
tb_vacinados = tb_vacinados.set_index('index')

# casos_confirmados = get_casos_confirmados()
casos_confirmados = pd.read_feather('casos_confirmados.feather')
casos_confirmados['faixa_etaria'] = [ faixa_etaria_labels[idx] for idx in np.digitize(casos_confirmados['idade'],faixa_etaria,right=False)]
casos_confirmados.groupby('evolucao')[['id']].count()

,id
evolucao,
ATIVO,297221
CURA,616543
OBITO,22783


In [3]:
print('casos_confirmados shape:', casos_confirmados.shape)
casos_confirmados.groupby('faixa_etaria')[['id']].count()

casos_confirmados shape: (936547, 20)


,id
faixa_etaria,
0-59,803395
60-69,79251
70-79,37521
80-89,13780
>=90,2600


In [4]:
obitos_confirmados = casos_confirmados.loc[(casos_confirmados['evolucao']=='OBITO')]
print('obitos_confirmados shape:', obitos_confirmados.shape)
obitos_confirmados.groupby('faixa_etaria')[['id']].count()

obitos_confirmados shape: (22783, 20)


,id
faixa_etaria,
0-59,6376
60-69,5798
70-79,5973
80-89,3708
>=90,928


In [5]:
casos_vacinados = casos_confirmados.loc[
    (casos_confirmados['cns'].notnull() & casos_confirmados['cns'].isin(tb_vacinados['cns'])) |
    (casos_confirmados['cpf'].notnull() & casos_confirmados['cpf'].isin(tb_vacinados['cpf'])) |
    (casos_confirmados['hash_mae'].notnull() & casos_confirmados['hash_mae'].isin(tb_vacinados['hash_mae'])) |
    (casos_confirmados['hash_nasc'].notnull() & casos_confirmados['hash_nasc'].isin(tb_vacinados['hash_nasc']))
]

print(len(casos_vacinados))

113145


In [6]:
tb_vacinados_cols = ['situacao','data_ultima_dose','vacina_nome','fabricante','data_aplicacao_1a_dose','data_aplicacao_2a_dose']

casos_vacinados = pd.merge(casos_confirmados.loc[casos_confirmados['cns'].notna()],tb_vacinados.loc[tb_vacinados['cns'].notna(),['cns']+tb_vacinados_cols], on='cns', how='inner')
tb_vacinados = tb_vacinados.drop(index=tb_vacinados.loc[tb_vacinados['cns'].isin(casos_vacinados['cns'])].index)
print(len(casos_vacinados))
print(len(tb_vacinados))

casos_vacinados = casos_vacinados.append(pd.merge(casos_confirmados.loc[casos_confirmados['cpf'].notna()],tb_vacinados.loc[tb_vacinados['cpf'].notna(),['cpf']+tb_vacinados_cols], on='cpf', how='inner'), ignore_index=True)
tb_vacinados = tb_vacinados.drop(index=tb_vacinados.loc[tb_vacinados['cpf'].isin(casos_vacinados['cpf'])].index)
print(len(casos_vacinados))
print(len(tb_vacinados))

casos_vacinados = casos_vacinados.append(pd.merge(casos_confirmados.loc[casos_confirmados['hash_mae'].notna()],tb_vacinados.loc[tb_vacinados['hash_mae'].notna(),['hash_mae']+tb_vacinados_cols], on='hash_mae', how='inner'), ignore_index=True)
tb_vacinados = tb_vacinados.drop(index=tb_vacinados.loc[tb_vacinados['hash_mae'].isin(casos_vacinados['hash_mae'])].index)
print(len(casos_vacinados))
print(len(tb_vacinados))

casos_vacinados = casos_vacinados.append(pd.merge(casos_confirmados.loc[casos_confirmados['hash_nasc'].notna()],tb_vacinados.loc[tb_vacinados['hash_nasc'].notna(),['hash_nasc']+tb_vacinados_cols], on='hash_nasc', how='inner'), ignore_index=True)
tb_vacinados = tb_vacinados.drop(index=tb_vacinados.loc[tb_vacinados['hash_nasc'].isin(casos_vacinados['hash_nasc'])].index)
print(len(casos_vacinados))
print(len(tb_vacinados))

print(len(casos_vacinados.loc[casos_vacinados.duplicated('id',keep=False)]))
casos_vacinados = casos_vacinados.drop_duplicates('id',keep='first')
len(casos_vacinados)

94409
1335497


In [11]:
casos_vacinados['diferenca_vacina_diag'] = casos_vacinados['data_diagnostico'] - casos_vacinados['data_ultima_dose']
casos_vacinados['diferenca_vacina_diag'] = casos_vacinados['diferenca_vacina_diag'].apply(lambda x: x.days)

# bins = [0,1,2,3,7,14,21,30,60,90]
# bins_label = ['antes da vacinação','hoje','24 horas','48 horas', '72 horas', '7 dias', '14 dias', '21 dias', '30 dias', '60 dias', '90 dias']

# diferenca_vacina_diag = casos_vacinados.groupby(['situacao',np.digitize(casos_vacinados['diferenca_vacina_diag'],bins)])[['paciente']].count().rename(columns={'paciente':'qtde'}).unstack().fillna(0).astype(int).T.droplevel(0,0)
# diferenca_vacina_diag.index = [ bins_label[x] for x in diferenca_vacina_diag.index ]
# diferenca_vacina_diag.columns.name = 'diferenca entre data_diagnostico e dt_vacina (em dias)'


# diferenca_vacina_diag

In [19]:
casos = pd.merge(casos_confirmados,casos_vacinados[['id','diferenca_vacina_diag']+tb_vacinados_cols],how='left',on='id')
casos['vacinado'] = 'NAO'
casos.loc[(casos['situacao']=='ambas doses aplicadas')&(casos['diferenca_vacina_diag']>14),'vacinado'] = 'SIM'


casos['ano_caso'] = casos['data_diagnostico'].apply(lambda x: x.year if isinstance(x,date) else 'erro')
casos['mes_caso'] = casos['data_diagnostico'].apply(lambda x: x.month if isinstance(x,date) else 'erro')
grupo = casos.groupby(['ano_caso','mes_caso','faixa_etaria','vacinado'])[['id']].count().unstack(0).unstack(0).unstack(-1)
grupo = grupo.droplevel(0,1)
grupo.index.name = None

grupo = grupo.fillna(0).astype(float)
grupo = grupo.append(pd.Series(grupo.sum(0),name='TOTAL'))

for idx in grupo.columns:
    if grupo.loc['TOTAL',idx] == 0:
        del grupo[idx] 

grupo.to_excel('casos_vacinados.xlsx')
grupo

ano_caso 2.020                            ... 2.021                        
mes_caso     3     4     5      6      7  ...     3      4            5    
vacinado   NAO   NAO   NAO    NAO    NAO  ...   SIM    NAO   SIM    NAO SIM
0-59       295 1.298 4.536 27.793 50.842  ...   951 72.012   874 14.648 191
60-69       29   112   393  2.265  4.492  ...    53  7.683    62  1.365   7
70-79       15    76   199  1.046  2.189  ...    44  3.431   103    364 103
80-89        7    35    96    462    908  ...    44    875   207     95  55
>=90         2     5    10     96    217  ...    30     76    89     17  14
TOTAL      348 1.526 5.234 31.662 58.648  ... 1.122 84.077 1.335 16.489 370

[6 rows x 19 columns]

In [20]:
obitos_vacinados = casos_vacinados.loc[casos_vacinados['evolucao']=='OBITO'].copy()
obitos_vacinados['diferenca_vacina_obito'] = obitos_vacinados['data_cura_obito'] - obitos_vacinados['data_ultima_dose']
obitos_vacinados['diferenca_vacina_obito'] = obitos_vacinados['diferenca_vacina_obito'].apply(lambda x: x.days)

# bins = [0,1,2,3,7,14,21,30,60,90]
# bins_label = ['antes da vacinação','hoje','24 horas','48 horas', '72 horas', '7 dias', '14 dias', '21 dias', '30 dias', '60 dias', '90 dias']

# diferenca_vacina_obito = obitos_vacinados.groupby(['situacao',np.digitize(obitos_vacinados['diferenca_vacina_obito'],bins)])[['paciente']].count().rename(columns={'paciente':'qtde'}).unstack().fillna(0).astype(int).T.droplevel(0,0)
# diferenca_vacina_obito.index = [ bins_label[x] for x in diferenca_vacina_obito.index ]
# diferenca_vacina_obito.columns.name = 'diferenca entre dt_obito e dt_vacina (em dias)'


# print(len(obitos_vacinados))
# diferenca_vacina_obito

In [21]:
obitos = pd.merge(obitos_confirmados,obitos_vacinados[['id','diferenca_vacina_diag']+tb_vacinados_cols],how='left',on='id')
obitos['vacinado'] = 'NAO'
obitos.loc[(obitos['situacao']=='ambas doses aplicadas')&(obitos['diferenca_vacina_diag']>14),'vacinado'] = 'SIM'


# obitos = obitos.loc[(obitos['data_cura_obito'] > '2021-01-18')&(obitos['idade']>=60)]
obitos['ano_obito'] = obitos['data_cura_obito'].apply(lambda x: x.year if isinstance(x,date) else 'erro')
obitos['mes_obito'] = obitos['data_cura_obito'].apply(lambda x: x.month if isinstance(x,date) else 'erro')
grupo = obitos.groupby(['ano_obito','mes_obito','faixa_etaria','vacinado'])[['id']].count().unstack(0).unstack(0).unstack(-1)
grupo = grupo.droplevel(0,1)
grupo.index.name = None

grupo = grupo.fillna(0).astype(float)
grupo = grupo.append(pd.Series(grupo.sum(0),name='TOTAL'))

for idx in grupo.columns:
    if grupo.loc['TOTAL',idx] == 0:
        del grupo[idx] 
    
grupo.to_excel('obitos_vacinados.xlsx')
grupo

ano_obito 2.020                                ... 2.021                            
mes_obito     3   4   5   6     7     8     9  ...     2     3         4       5    
vacinado    NAO NAO NAO NAO   NAO   NAO   NAO  ...   NAO   NAO SIM   NAO SIM NAO SIM
0-59          2  19  22 127   353   331   256  ...   476 1.948   1 1.318   4 224   1
60-69         2   6  24 129   287   341   244  ...   473 1.544   1 1.088   2 160   1
70-79         0  12  24 152   359   382   321  ...   550 1.517   3   902   6  86   2
80-89         1  13  16 122   257   266   199  ...   337   877   5   349  31  38   9
>=90          0   5   2  27    80    80    80  ...    88   191   1    38  16   4   6
TOTAL         5  55  88 557 1.336 1.400 1.100  ... 1.924 6.077  11 3.695  59 512  19

[6 rows x 18 columns]

In [22]:
casos2020 = casos.loc[(casos['ano_caso']==2020)&(casos['mes_caso'].isin([9,10,11,12]))]
casos2020 = casos2020.groupby('faixa_etaria')[['id']].count()
total = pd.Series(casos2020.sum(),name='TOTAL')
casos2020 = casos2020.append(total)
casos2020['%**'] = casos2020.apply(lambda x: x['id']/total, axis=1)
casos2020.to_excel('casos2020.xlsx')
casos2020

,id,%**
faixa_etaria,,
0-59,268788,1
60-69,25250,0
70-79,11690,0
80-89,4349,0
>=90,824,0
TOTAL,310901,1


In [23]:
casos2021 = casos.loc[(casos['ano_caso']==2021)&(casos['mes_caso'].isin([1,2,3,4]))]
casos2021 = casos2021.groupby(['faixa_etaria','vacinado'])[['id']].count().unstack()
total = pd.Series(casos2021.sum(),name='TOTAL')
casos2021 = casos2021.append(total)
casos2021['TOTAL'] = casos2021.sum(1)
casos2021[('TOTAL','%**')] = 0#casos2021['TOTAL'].apply(lambda x: x.iloc[0]/total, 1)
casos2021.to_excel('casos2021.xlsx')
casos2021

id         TOTAL    
vacinado         NAO   SIM         %**
faixa_etaria                          
0-59          384910  1830  386740   0
60-69          40833   115   40948   0
70-79          19627   147   19774   0
80-89           6650   251    6901   0
>=90            1086   119    1205   0
TOTAL         453106  2462  455568   0

In [24]:
obitos2020 = obitos.loc[(obitos['ano_obito']==2020)&(obitos['mes_obito'].isin([9,10,11,12]))]
obitos2020 = obitos2020.groupby('faixa_etaria')[['id']].count()
total = pd.Series(obitos2020.sum(),name='TOTAL')
obitos2020 = obitos2020.append(total)
obitos2020['%**'] = obitos2020.apply(lambda x: x['id']/total, axis=1)
obitos2020.to_excel('obitos2020.xlsx')
obitos2020

,id,%**
faixa_etaria,,
0-59,1132,0
60-69,1251,0
70-79,1449,0
80-89,1010,0
>=90,279,0
TOTAL,5121,1


In [25]:
obitos2021 = obitos.loc[(obitos['ano_obito']==2021)&(obitos['mes_obito'].isin([1,2,3,4]))]
obitos2021 = obitos2021.groupby(['faixa_etaria','vacinado'])[['id']].count().unstack()
total = pd.Series(obitos2021.sum(),name='TOTAL')
obitos2021 = obitos2021.append(total)
obitos2021['TOTAL'] = obitos2021.sum(1)
obitos2021[('TOTAL','%**')] = 0#obitos2021['TOTAL'].apply(lambda x: x.iloc[0]/total, 1)
obitos2021.to_excel('obitos2021.xlsx')
casos2021

id         TOTAL    
vacinado         NAO   SIM         %**
faixa_etaria                          
0-59          384910  1830  386740   0
60-69          40833   115   40948   0
70-79          19627   147   19774   0
80-89           6650   251    6901   0
>=90            1086   119    1205   0
TOTAL         453106  2462  455568   0